# Online Transaction Fraud Detection
### - This notebook shows an approach to classify the online transaction as fraudulant or genuine. I have covered memory reduction algorithm without data loss, handling missing data and feature engineering in this notebook.
### - Memory reduction can be used for any other problem while handling missing data and feature engineering is specific to data and problem at hand. 
### - Time series nature of data - With time there are new users transactions in the test data which are  not present in the train data. Also the transaction pattern changes with the months e.g. electricty bill payment increases in winter as compared to summer. I have tried to make the data independent of the users and purely on the transactions.
### - This data contains huge amount of NAN values. Way of dealing with these values do affect the performance. I have tried multiple ways dealing with the data and are  explained in the relevant section below.
### - Class Imbalance - Data is hugely skewed and has about 2% positive class. Ways to work on imbalanced classes were tried however Lightgbm handles class imbalance and method to handle class imbalance does not affect the performance of Lightgbm. However for other models class imbalance needs to be handled. 
### - Data consists of more than 400 columns - EDA is very crucial for selecting the columns. Extensive EDA is performed in seperate notebook. This notebook focuses on feature generation. 

In [ ]:
# The code was removed by Watson Studio for sharing.

## Import the required libraries

In [ ]:
import catboost
from catboost import CatBoostClassifier, Pool, cv
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
pd.set_option("display.max_columns", 400)

## Load Data
### Data consists train transaction details, test transaction details, train identity details, test identity details, 

In [ ]:
# The code was removed by Watson Studio for sharing.

In [ ]:
# The code was removed by Watson Studio for sharing.

In [ ]:
# The code was removed by Watson Studio for sharing.

In [ ]:
# The code was removed by Watson Studio for sharing.

In [ ]:
print(df_train.shape)
print(df_train_identity.shape)
print(df_test.shape)
print(df_test_identity.shape)

### Removing the columns which has only one unique value as they will not add any value to our model

In [ ]:
count=0
for col in df_train.columns[2:]:
    if len(df_train[col].unique())==1 or len(df_test[col].unique())==1:
        df_train.drop([col], axis=1,inplace=True)
        df_test.drop([col], axis=1,inplace=True)
        count=count+1
        print(col)
count

### Merging the Transaction and identity data

In [ ]:
df_train = pd.merge(df_train, df_train_identity, how='left', left_on=['TransactionID'], right_on=['TransactionID'], copy=False)
print('train done')
df_test = pd.merge(df_test, df_test_identity, how='left', left_on=['TransactionID'], right_on=['TransactionID'], copy=False)
print('test done')
df_train_identity = 0
df_test_identity = 0
del df_train_identity
del df_test_identity

## Handling missing data
### - This is an example of imbalanced classes and hence deleting the rows with missing data is not fruitful. Replacing the missing values with mean is also not feasible as the details of the columns are also masked. 
### - I tried following ways to handle the missing data
### 1. Replacing the missing values by large negative number like -999 so that Lightgbm treat these values seperately rather than ignoring them. However the problem with this approach was that there are few columns with negative values (-182). For these columns replacing NAN with -999 showed decrease in performance as our model merged them with less than 0 class. After analyzing these columns it was found out that keeping these values 0 was better choice rather than mean or -999. (implemented below)
### 2. Another approach which I experimented is not suggested for all the models or problems. Since I was using lightgbm for classification with large number of categorical data, this approach worked. I made the missing values as another class of values 0. So I made sure that there are no 0 values in the column containing the missing data. Since most of the columns are categorical increasing the values by 1 and replacing the missing data with 0 did not harm the performance rather improved it. (not implemented in this notebook)

In [ ]:
df_train = df_train.replace([np.inf,-np.inf], np.nan)
df_test = df_test.replace([np.inf,-np.inf], np.nan)


In [ ]:
# ['id_01','id_03','id_04','id_05','id_06','id_07','id_08','id_09','id_10','id_14'] are categorical and hence missing values are not replaced by 0
# other values where there are negative values in columns, missing data is replaced by 0
for col in df_train.columns[2:]:
    if df_train[col].dtype in ['int64','float64']:
        if (df_train[col].min()<0 or df_test[col].min()<0) and col not in ['id_01','id_03','id_04','id_05','id_06','id_07','id_08','id_09','id_10','id_14']:
            print(col)
            df_train[col] =df_train[col].replace(np.nan, 0)
            df_test[col] = df_test[col].replace(np.nan, 0)


In [ ]:
# Replacing the missing data for rest of the columns with -999
df_train =df_train.replace(np.nan, -999)
df_test = df_test.replace(np.nan, -999)


## Reducing memory
### Default datatype is float64 even for integer values as the nan values are considered as float in python.
### find out the columns with only integer data and change the datatype. This results in more than 60% reduction in memory usage

In [ ]:
for col in df_train.columns[2:]:
    if df_train[col].dtype in ['float64']:
        sum_before = df_train[col].sum() # used to make sure there is no loss of data
        is_int = 0
        for val in df_train[col].unique(): # used to check if all the values are integers 
            if val.is_integer()==False:
                is_int=1
                break
        if is_int==0:
            if np.abs(df_train[col]).max()<127 and np.abs(df_test[col]).max()<127:
                print('8 ',col)
                df_train[col] = df_train[col].astype('int8', inplace=True)
                df_test[col] = df_test[col].astype('int8', inplace=True)
                sum_after = df_train[col].sum()
                print(sum_before-sum_after)
                continue
                
            if np.abs(df_train[col]).max()<32767 and np.abs(df_test[col]).max()<32767 : 
                print('16 ',col)
                df_train[col] = df_train[col].astype('int16', inplace=True)
                df_test[col] = df_test[col].astype('int16', inplace=True)
                sum_after = df_train[col].sum()
                print(sum_before-sum_after)
                continue
            if np.abs(df_train[col]).max()<2147483647 and np.abs(df_test[col]).max()<2147483647 : 
                print('32 ',col)
                df_train[col] = df_train[col].astype('int32', inplace=True)
                df_test[col] = df_test[col].astype('int32', inplace=True)
                sum_after = df_train[col].sum()
                print(sum_before-sum_after)
                continue
          

    elif df_train[col].dtype in ['int64']:
        sum_before = df_train[col].sum()
        if np.abs(df_train[col]).max()<127 and np.abs(df_test[col]).max()<127 : 
            print('8 ',col)
            df_train[col] = df_train[col].astype('int8', inplace=True)
            df_test[col] = df_test[col].astype('int8', inplace=True)
            sum_after = df_train[col].sum()
            print(sum_before-sum_after)
            continue
        if np.abs(df_train[col]).max()<32767 and np.abs(df_test[col]).max()<32767: 
            print('16 ', col)
            df_train[col] = df_train[col].astype('int16', inplace=True)
            df_test[col] = df_test[col].astype('int16', inplace=True)
            sum_after = df_train[col].sum()
            print(sum_before-sum_after)
            continue
        if np.abs(df_train[col]).max()<2147483647 and np.abs(df_test[col]).max()<2147483647: 
            print('32 ',col)
            df_train[col] = df_train[col].astype('int32', inplace=True)
            df_test[col] = df_test[col].astype('int32', inplace=True)      
            sum_after = df_train[col].sum()
            print(sum_before-sum_after)

### Converting non-numercal data to categorical
### we have taken care of missing data in numerical columns. Below mentioned columns are categorical and hence replacing the missing data with -999 or 0 does not make any difference for  my lightgbm model

In [ ]:
df_train['card4'].replace([np.nan, 'discover', 'mastercard' ,'visa' ,'american express'],[0,1,2,3,4], inplace=True)
df_train['card4']=df_train['card4'].astype('category')
df_train['card6'].replace([np.nan, 'credit', 'debit', 'debit or credit', 'charge card'],[0,1,2,3,4], inplace=True)
df_train['card6']=df_train['card6'].astype('category')
df_train['M1'].replace([np.nan, 'F','T'],[0,1,2], inplace=True)
df_train['M1']=df_train['M1'].astype('category')
df_train['M2'].replace([np.nan, 'F','T'],[0,1,2], inplace=True)
df_train['M2']=df_train['M2'].astype('category')
df_train['M3'].replace([np.nan, 'F','T'],[0,1,2], inplace=True)
df_train['M3']=df_train['M3'].astype('category')
df_train['M4'].replace([np.nan, 'M2', 'M0','M1'],[0,1,2,3], inplace=True)
df_train['M4']=df_train['M4'].astype('category')
df_train['M5'].replace([np.nan, 'F','T'],[0,1,2], inplace=True)
df_train['M5']=df_train['M5'].astype('category')
df_train['M6'].replace([np.nan, 'F','T'],[0,1,2], inplace=True)
df_train['M6']=df_train['M6'].astype('category')
df_train['M7'].replace([np.nan, 'F','T'],[0,1,2], inplace=True)
df_train['M7']=df_train['M7'].astype('category')
df_train['M8'].replace([np.nan, 'F','T'],[0,1,2], inplace=True)
df_train['M8']=df_train['M8'].astype('category')
df_train['M9'].replace([np.nan, 'F','T'],[0,1,2], inplace=True)
df_train['M9']=df_train['M9'].astype('category')
df_train['id_12'].replace([np.nan, 'NotFound','Found'],[0,1,2], inplace=True)
df_train['id_12']=df_train['id_12'].astype('category')
df_train['id_15'].replace([np.nan, 'New', 'Found', 'Unknown'],[0,1,2,3], inplace=True)
df_train['id_15']=df_train['id_15'].astype('category')
df_train['id_16'].replace([np.nan, 'NotFound','Found'],[0,1,2], inplace=True)
df_train['id_16']=df_train['id_16'].astype('category')
df_train['id_23'].replace([np.nan, 'IP_PROXY:TRANSPARENT', 'IP_PROXY:ANONYMOUS', 'IP_PROXY:HIDDEN'],[0,1,2,3], inplace=True)
df_train['id_23']=df_train['id_23'].astype('category')
df_train['id_27'].replace([np.nan, 'NotFound','Found'],[0,1,2], inplace=True)
df_train['id_27']=df_train['id_27'].astype('category')
df_train['id_28'].replace([np.nan, 'New','Found'],[0,1,2], inplace=True)
df_train['id_28']=df_train['id_28'].astype('category')
df_train['id_29'].replace([np.nan, 'NotFound','Found'],[0,1,2], inplace=True)
df_train['id_29']=df_train['id_29'].astype('category')
df_train['id_34'].replace([np.nan, 'match_status:2', 'match_status:1', 'match_status:0' ,'match_status:-1'],[0,1,2,3,4], inplace=True)
df_train['id_34']=df_train['id_34'].astype('category')
df_train['id_35'].replace([np.nan, 'T','F'],[0,1,2], inplace=True)
df_train['id_35']=df_train['id_35'].astype('category')
df_train['id_36'].replace([np.nan, 'T','F'],[0,1,2], inplace=True)
df_train['id_36']=df_train['id_36'].astype('category')
df_train['id_37'].replace([np.nan, 'T','F'],[0,1,2], inplace=True)
df_train['id_37']=df_train['id_37'].astype('category')
df_train['id_38'].replace([np.nan, 'T','F'],[0,1,2], inplace=True)
df_train['id_38']=df_train['id_38'].astype('category')
df_train['DeviceType'].replace([np.nan, 'mobile' ,'desktop'],[0,1,2], inplace=True)
df_train['DeviceType']=df_train['DeviceType'].astype('category')
df_train['ProductCD'].replace(['W', 'H', 'C', 'S', 'R'],[1,2,3,4,5], inplace=True)
df_train['ProductCD']=df_train['ProductCD'].astype('category')

df_test['ProductCD'].replace(['W', 'H', 'C', 'S', 'R'],[1,2,3,4,5], inplace=True)
df_test['ProductCD']=df_test['ProductCD'].astype('category')
df_test['card4'].replace([np.nan, 'discover', 'mastercard' ,'visa' ,'american express'],[0,1,2,3,4], inplace=True)
df_test['card4']=df_test['card4'].astype('category')
df_test['card6'].replace([np.nan, 'credit', 'debit', 'debit or credit', 'charge card'],[0,1,2,3,4], inplace=True)
df_test['card6']=df_test['card6'].astype('category')
df_test['M1'].replace([np.nan, 'F','T'],[0,1,2], inplace=True)
df_test['M1']=df_test['M1'].astype('category')
df_test['M2'].replace([np.nan, 'F','T'],[0,1,2], inplace=True)
df_test['M2']=df_test['M2'].astype('category')
df_test['M3'].replace([np.nan, 'F','T'],[0,1,2], inplace=True)
df_test['M3']=df_test['M3'].astype('category')
df_test['M4'].replace([np.nan, 'M2', 'M0','M1'],[0,1,2,3], inplace=True)
df_test['M4']=df_test['M4'].astype('category')
df_test['M5'].replace([np.nan, 'F','T'],[0,1,2], inplace=True)
df_test['M5']=df_test['M5'].astype('category')
df_test['M6'].replace([np.nan, 'F','T'],[0,1,2], inplace=True)
df_test['M6']=df_test['M6'].astype('category')
df_test['M7'].replace([np.nan, 'F','T'],[0,1,2], inplace=True)
df_test['M7']=df_test['M7'].astype('category')
df_test['M8'].replace([np.nan, 'F','T'],[0,1,2], inplace=True)
df_test['M8']=df_test['M8'].astype('category')
df_test['M9'].replace([np.nan, 'F','T'],[0,1,2], inplace=True)
df_test['M9']=df_test['M9'].astype('category')
df_test['id_12'].replace([np.nan, 'NotFound','Found'],[0,1,2], inplace=True)
df_test['id_12']=df_test['id_12'].astype('category')
df_test['id_15'].replace([np.nan, 'New', 'Found', 'Unknown'],[0,1,2,3], inplace=True)
df_test['id_15']=df_test['id_15'].astype('category')
df_test['id_16'].replace([np.nan, 'NotFound','Found'],[0,1,2], inplace=True)
df_test['id_16']=df_test['id_16'].astype('category')
df_test['id_23'].replace([np.nan, 'IP_PROXY:TRANSPARENT', 'IP_PROXY:ANONYMOUS', 'IP_PROXY:HIDDEN'],[0,1,2,3], inplace=True)
df_test['id_23']=df_test['id_23'].astype('category')
df_test['id_27'].replace([np.nan, 'NotFound','Found'],[0,1,2], inplace=True)
df_test['id_27']=df_test['id_27'].astype('category')
df_test['id_28'].replace([np.nan, 'New','Found'],[0,1,2], inplace=True)
df_test['id_28']=df_test['id_28'].astype('category')
df_test['id_29'].replace([np.nan, 'NotFound','Found'],[0,1,2], inplace=True)
df_test['id_29']=df_test['id_29'].astype('category')
df_test['id_34'].replace([np.nan, 'match_status:2', 'match_status:1', 'match_status:0' ,'match_status:-1'],[0,1,2,3,4], inplace=True)
df_test['id_34']=df_test['id_34'].astype('category')
df_test['id_35'].replace([np.nan, 'T','F'],[0,1,2], inplace=True)
df_test['id_35']=df_test['id_35'].astype('category')
df_test['id_36'].replace([np.nan, 'T','F'],[0,1,2], inplace=True)
df_test['id_36']=df_test['id_36'].astype('category')
df_test['id_37'].replace([np.nan, 'T','F'],[0,1,2], inplace=True)
df_test['id_37']=df_test['id_37'].astype('category')
df_test['id_38'].replace([np.nan, 'T','F'],[0,1,2], inplace=True)
df_test['id_38']=df_test['id_38'].astype('category')
df_test['DeviceType'].replace([np.nan, 'mobile' ,'desktop'],[0,1,2], inplace=True)
df_test['DeviceType']=df_test['DeviceType'].astype('category')



In [ ]:
# saving the data
project.save_data(data=df_train.to_csv(index=False),file_name='train_merged_999_new.csv',overwrite=True)
print('train done')
project.save_data(data=df_test.to_csv(index=False),file_name='test_merged_999_new.csv',overwrite=True)

### Extracting the month, day, week from 'TransactionDT'

In [ ]:
import datetime

In [ ]:
START_DATE = '2017-12-01'
startdate = datetime.datetime.strptime(START_DATE, '%Y-%m-%d')
df_temp = pd.concat([df_train[['TransactionID','TransactionDT']],df_test[['TransactionID','TransactionDT']]],copy=False)

df_temp['TransactionDT'] = df_temp['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds = x)))

df_temp['year'] = df_temp['TransactionDT'].dt.year
df_temp['month'] = df_temp['TransactionDT'].dt.month
df_temp['dow'] = df_temp['TransactionDT'].dt.dayofweek
df_temp['hour'] = df_temp['TransactionDT'].dt.hour
df_temp['day'] = df_temp['TransactionDT'].dt.day

df_temp = df_temp.drop(['TransactionDT'],axis=1)

df_train = pd.merge(df_train, df_temp, how='left', left_on=['TransactionID'], right_on=['TransactionID'], copy=False)
print('train done')
df_test = pd.merge(df_test, df_temp, how='left', left_on=['TransactionID'], right_on=['TransactionID'], copy=False)
print('test done')

### slicing out piece of data for testing and validation as we do not have actual classes of given test data for evaluting our model
### Kindly note that df_valid is going to be used for evaluating the model.
### Since I want my model to be independent of time, I am dividing my test and train data based on months.

In [ ]:
df_valid = df_train[df_train['month']==5]
df_train = df_train[df_train['month']!=5]

### Slicing out the columns required for feature engineering as other columns will only slow down the execution

In [ ]:
df_train = df_train[['TransactionID', 'isFraud', 'TransactionDT', 'TransactionAmt',
       'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6',
       'addr1', 'addr2', 'dist1', 'dist2','month','hour','day','dow']]
df_valid = df_valid[['TransactionID', 'isFraud', 'TransactionDT', 'TransactionAmt',
       'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6',
       'addr1', 'addr2', 'dist1', 'dist2','month','hour','day','dow']]
df_test = df_test[['TransactionID', 'TransactionDT', 'TransactionAmt',
       'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6',
       'addr1', 'addr2', 'dist1', 'dist2','month','hour','day','dow']]

In [ ]:
# Saving the data
project.save_data(data=df_train.to_csv(index=False),file_name='train_short_new.csv',overwrite=True)
print('train done')
project.save_data(data=df_valid.to_csv(index=False),file_name='valid_short_new.csv',overwrite=True)
print('valid done')
project.save_data(data=df_test.to_csv(index=False),file_name='test_short_new.csv',overwrite=True)

In [ ]:
# We are going to divide by hours for feature engineering and hence to avoid nan values we are adding 1 to hour column
df_train['hour'] = df_train['hour'] +1
df_valid['hour'] = df_valid['hour'] +1
df_test['hour'] = df_test['hour'] +1

### Data does not have unique identifier for the user. unique identifier 'uid1' is created below by combining all the card types and address columns. This uid is used after experimenting with different combinations

In [ ]:
df_train['uid1'] = df_train['card1'].astype(str)+'_'+df_train['card2'].astype(str)+'_'+df_train['card3'].astype(str)+'_'+df_train['card4'].astype(str)+'_'+df_train['card5'].astype(str)+'_'+df_train['card6'].astype(str)+'_'+df_train['addr1'].astype(str)+'_'+df_train['addr2'].astype(str)
df_valid['uid1'] = df_valid['card1'].astype(str)+'_'+df_valid['card2'].astype(str)+'_'+df_valid['card3'].astype(str)+'_'+df_valid['card4'].astype(str)+'_'+df_valid['card5'].astype(str)+'_'+df_valid['card6'].astype(str)+'_'+df_valid['addr1'].astype(str)+'_'+df_valid['addr2'].astype(str)
df_test['uid1'] = df_test['card1'].astype(str)+'_'+df_test['card2'].astype(str)+'_'+df_test['card3'].astype(str)+'_'+df_test['card4'].astype(str)+'_'+df_test['card5'].astype(str)+'_'+df_test['card6'].astype(str)+'_'+df_test['addr1'].astype(str)+'_'+df_test['addr2'].astype(str)

### Data consists for cards which are frequently used with large number of transactions to compare to. However there are cards and uid1 where there are very few transactions. To identify such cards, we have created seperate column 'rare_cards' where the cards are rarely used.

In [ ]:
# Finding number of times a card is used based on card type and the address columns

columns = ['TransactionID', 'TransactionDT', 'TransactionAmt',
       'ProductCD', 'addr1']
df_temp=pd.concat([df_train[columns],df_valid[columns],df_test[columns]],copy=False)
df_addr1_count = df_temp['addr1'].value_counts().reset_index().rename(columns={'index':'addr1','addr1':'addr1_count'})
df_train = pd.merge(df_train, df_addr1_count[['addr1','addr1_count']], how='left', left_on=['addr1'], right_on=['addr1'], copy=False)
df_test = pd.merge(df_test, df_addr1_count[['addr1','addr1_count']], how='left', left_on=['addr1'], right_on=['addr1'], copy=False)
df_valid = pd.merge(df_valid, df_addr1_count[['addr1','addr1_count']], how='left', left_on=['addr1'], right_on=['addr1'], copy=False)

columns = ['TransactionID', 'TransactionDT', 'TransactionAmt',
       'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6','uid','uid1']
df_temp=pd.concat([df_train[columns],df_valid[columns],df_test[columns]],copy=False)
df_card1_count = df_temp['card1'].value_counts().reset_index().rename(columns={'index':'card1','card1':'card1_count'})
df_train = pd.merge(df_train, df_card1_count[['card1','card1_count']], how='left', left_on=['card1'], right_on=['card1'], copy=False)
df_test = pd.merge(df_test, df_card1_count[['card1','card1_count']], how='left', left_on=['card1'], right_on=['card1'], copy=False)
df_valid = pd.merge(df_valid, df_card1_count[['card1','card1_count']], how='left', left_on=['card1'], right_on=['card1'], copy=False)

df_card2_count = df_temp['card2'].value_counts().reset_index().rename(columns={'index':'card2','card2':'card2_count'})
df_train = pd.merge(df_train, df_card2_count[['card2','card2_count']], how='left', left_on=['card2'], right_on=['card2'], copy=False)
df_test = pd.merge(df_test, df_card2_count[['card2','card2_count']], how='left', left_on=['card2'], right_on=['card2'], copy=False)
df_valid = pd.merge(df_valid, df_card2_count[['card2','card2_count']], how='left', left_on=['card2'], right_on=['card2'], copy=False)

df_card3_count = df_temp['card3'].value_counts().reset_index().rename(columns={'index':'card3','card3':'card3_count'})
df_train = pd.merge(df_train, df_card3_count[['card3','card3_count']], how='left', left_on=['card3'], right_on=['card3'], copy=False)
df_test = pd.merge(df_test, df_card3_count[['card3','card3_count']], how='left', left_on=['card3'], right_on=['card3'], copy=False)
df_valid = pd.merge(df_valid, df_card3_count[['card3','card3_count']], how='left', left_on=['card3'], right_on=['card3'], copy=False)

df_card5_count = df_temp['card5'].value_counts().reset_index().rename(columns={'index':'card5','card5':'card5_count'})
df_train = pd.merge(df_train, df_card5_count[['card5','card5_count']], how='left', left_on=['card5'], right_on=['card5'], copy=False)
df_test = pd.merge(df_test, df_card5_count[['card5','card5_count']], how='left', left_on=['card5'], right_on=['card5'], copy=False)
df_valid = pd.merge(df_valid, df_card5_count[['card5','card5_count']], how='left', left_on=['card5'], right_on=['card5'], copy=False)

df_card6_count = df_temp['card6'].value_counts().reset_index().rename(columns={'index':'card6','card6':'card6_count'})
df_train = pd.merge(df_train, df_card6_count[['card6','card6_count']], how='left', left_on=['card6'], right_on=['card6'], copy=False)
df_test = pd.merge(df_test, df_card6_count[['card6','card6_count']], how='left', left_on=['card6'], right_on=['card6'], copy=False)
df_valid = pd.merge(df_valid, df_card6_count[['card6','card6_count']], how='left', left_on=['card6'], right_on=['card6'], copy=False)

df_uid1_count = df_temp['uid1'].value_counts().reset_index().rename(columns={'index':'uid1','uid1':'uid1_count'})
df_train = pd.merge(df_train, df_uid1_count[['uid1','uid1_count']], how='left', left_on=['uid1'], right_on=['uid1'], copy=False)
df_test = pd.merge(df_test, df_uid1_count[['uid1','uid1_count']], how='left', left_on=['uid1'], right_on=['uid1'], copy=False)
df_valid = pd.merge(df_valid, df_uid1_count[['uid1','uid1_count']], how='left', left_on=['uid1'], right_on=['uid1'], copy=False)


In [ ]:
# Finding out rarely uses cards or cards which does not have large usage history

df_temp = pd.concat([df_train[['TransactionID','card1_count','card2_count','card3_count','card5_count','card6_count','addr1_count','uid_count','uid1_count']],
                     df_valid[['TransactionID','card1_count','card2_count','card3_count','card5_count','card6_count','addr1_count','uid_count','uid1_count']],
                     df_test[['TransactionID','card1_count','card2_count','card3_count','card5_count','card6_count','addr1_count','uid_count','uid1_count']]], copy=False)
rare_card1 = np.zeros(df_temp.shape[0], 'int8')
rare_card2 = np.zeros(df_temp.shape[0], 'int8')
rare_card3 = np.zeros(df_temp.shape[0], 'int8')
rare_card5 = np.zeros(df_temp.shape[0], 'int8')
rare_card6 = np.zeros(df_temp.shape[0], 'int8')
rare_addr1 = np.zeros(df_temp.shape[0], 'int8')
rare_uid1 = np.zeros(df_temp.shape[0], 'int8')

card1_list = df_temp['card1_count'].values
card2_list = df_temp['card2_count'].values
card3_list = df_temp['card3_count'].values
card5_list = df_temp['card5_count'].values
card6_list = df_temp['card6_count'].values
addr1_list = df_temp['addr1_count'].values
uid1_list = df_temp['uid1_count'].values
                    
for i in range(0,df_temp.shape[0]):
    if card1_list[i]<10:
        rare_card1[i] = 1
    if card2_list[i]<10:
        rare_card2[i] = 1
    if card3_list[i]<10:
        rare_card3[i] = 1
    if card5_list[i]<10:
        rare_card5[i] = 1
    if card6_list[i]<10:
        rare_card6[i] = 1
    if addr1_list[i]<10:
        rare_addr1[i] = 1
    if uid_list[i]<10:
        rare_uid[i] = 1
    if uid1_list[i]<10:
        rare_uid1[i] = 1

df_temp['rare_card1'] = rare_card1
df_temp['rare_card2'] = rare_card2
df_temp['rare_card3'] = rare_card3
df_temp['rare_card5'] = rare_card5
df_temp['rare_card6'] = rare_card6
df_temp['rare_addr1'] = rare_addr1
df_temp['rare_uid'] = rare_uid
df_temp['rare_uid1'] = rare_uid1

df_temp = df_temp.drop(['card1_count','card2_count','card3_count','card5_count','card6_count','addr1_count','uid_count','uid1_count'], axis=1)

df_train = pd.merge(df_train, df_temp, how='left', left_on=['TransactionID'], right_on=['TransactionID'], copy=False)
df_test = pd.merge(df_test, df_temp, how='left', left_on=['TransactionID'], right_on=['TransactionID'], copy=False)
df_valid = pd.merge(df_valid, df_temp, how='left', left_on=['TransactionID'], right_on=['TransactionID'], copy=False)


### There are many instances where the fraudalant transaction occur in burst mode. i.e. large number of transactions in short amount of time
### Two new features are generated to understand this behaviour
### 1. Nature of transaction in last 1 hour - This even helps to undersatnd is the card is frequently used in single hour like corporate cards
### 2. Details of last transaction irrespective of last one hour


In [ ]:
# this code generates details of transaction in last one hour from the given transaction
df_temp = pd.concat([df_train[['TransactionID', 'TransactionDT', 'TransactionAmt','ProductCD','addr1', 'addr2', 'dist1', 'dist2', 'uid1']],
                    df_valid[['TransactionID', 'TransactionDT', 'TransactionAmt','ProductCD','addr1', 'addr2', 'dist1', 'dist2', 'uid1']],
                    df_test[['TransactionID', 'TransactionDT', 'TransactionAmt','ProductCD','addr1', 'addr2', 'dist1', 'dist2', 'uid1']]],
                    copy=False)

df_temp = df_temp.sort_values('TransactionDT')

last_time = np.zeros(df_temp.shape[0], 'int32')
last_amt = np.zeros(df_temp.shape[0],'float32')
last_prod = np.zeros(df_temp.shape[0],'int8')
last_count = np.zeros(df_temp.shape[0],'int32')
last_dist1 = np.zeros(df_temp.shape[0],'int32')
last_dist2 = np.zeros(df_temp.shape[0],'int32')

time_list = df_temp['TransactionDT'].values # using np arrays rather than dataframe speeds up the execution
amt_list = df_temp['TransactionAmt'].values
prod_list = df_temp['ProductCD'].values
dist1_list = df_temp['dist1'].values
dist2_list = df_temp['dist2'].values
uid1_list = df_temp['uid1'].values

for i in range(0,df_temp.shape[0]):
    index = i-1
    time_i = time_list[i]
    amt_i = amt_list[i]
    prod_i = prod_list[i]
    dist1_i = dist1_list[i]
    dist2_i = dist2_list[i]
    uid1_i = uid1_list[i]
    
    index_track = -1
    count=0
    while (index>0 and (time_i-time_list[index])<3601):
        if uid1_i == uid1_list[index]:
            count=count + 1
            index_track = index
            
        index = index -1
    if count > 0:
        last_time[i] = time_i - time_list[index_track]
        last_amt[i] = (amt_i - amt_list[index_track])/amt_list[index_track]
        last_prod[i] = prod_i - prod_list[index_track]
        last_dist1[i] = dist1_i - time_list[index_track]
        last_dist2[i] = dist2_i - time_list[index_track]
        last_count[i] = count
    
    print(i,count)
    
df_temp['last_time_uid1_all'] = last_time
df_temp['last_amt_uid1_all']= last_amt
df_temp['last_prod_uid1_all'] = last_prod
df_temp['last_dist1_uid1_all'] = last_dist1 
df_temp['last_dist2_uid1_all'] = last_dist2
df_temp['last_count_uid1_all'] = last_count

df_temp = df_temp.drop(['TransactionDT', 'TransactionAmt','ProductCD','addr1', 'addr2', 'dist1', 'dist2', 'uid1'], axis=1)

df_train = pd.merge(df_train, df_temp, how='left', left_on=['TransactionID'], right_on=['TransactionID'], copy=False)
df_test = pd.merge(df_test, df_temp, how='left', left_on=['TransactionID'], right_on=['TransactionID'], copy=False)
df_valid = pd.merge(df_valid, df_temp, how='left', left_on=['TransactionID'], right_on=['TransactionID'], copy=False)


In [ ]:
# This code generates details of last transaction
df_temp = pd.concat([df_train[['TransactionID', 'TransactionDT', 'TransactionAmt','ProductCD','addr1', 'addr2', 'dist1', 'dist2', 'uid1']],
                    df_valid[['TransactionID', 'TransactionDT', 'TransactionAmt','ProductCD','addr1', 'addr2', 'dist1', 'dist2', 'uid1']],
                    df_test[['TransactionID', 'TransactionDT', 'TransactionAmt','ProductCD','addr1', 'addr2', 'dist1', 'dist2', 'uid1']]],
                    copy=False)

df_temp = df_temp.sort_values('TransactionDT')

last_time = np.zeros(df_temp.shape[0], 'int32')
last_amt = np.zeros(df_temp.shape[0],'float32')
last_prod = np.zeros(df_temp.shape[0],'int8')
last_dist1 = np.zeros(df_temp.shape[0],'int32')
last_dist2 = np.zeros(df_temp.shape[0],'int32')

time_list = df_temp['TransactionDT'].values
amt_list = df_temp['TransactionAmt'].values
prod_list = df_temp['ProductCD'].values
dist1_list = df_temp['dist1'].values
dist2_list = df_temp['dist2'].values
uid1_list = df_temp['uid1'].values

for i in range(0,df_temp.shape[0]):
    index = i-1
    time_i = time_list[i]
    amt_i = amt_list[i]
    prod_i = prod_list[i]
    dist1_i = dist1_list[i]
    dist2_i = dist2_list[i]
    uid1_i = uid1_list[i]
    
    count=0
    while (index>0):
        if uid1_i == uid1_list[index]:
            count=count + 1
            
            last_time[i] = time_i - time_list[index]
            last_amt[i] = (amt_i - amt_list[index])/amt_list[index]
            last_prod[i] = prod_i - prod_list[index]
            last_dist1[i] = dist1_i - time_list[index]
            last_dist2[i] = dist2_i - time_list[index]
            last_count[i] = count
            print(i,count)        
            break
        index = index -1
        
    
    
df_temp['last_time_uid1_overall'] = last_time
df_temp['last_amt_uid1_overall']= last_amt
df_temp['last_prod_uid1_overall'] = last_prod
df_temp['last_dist1_uid1_overall'] = last_dist1 
df_temp['last_dist2_uid1_overall'] = last_dist2
df_temp['last_count_uid1_overall'] = last_count

df_temp = df_temp.drop(['TransactionDT', 'TransactionAmt','ProductCD','addr1', 'addr2', 'dist1', 'dist2', 'uid1'], axis=1)

df_train = pd.merge(df_train, df_temp, how='left', left_on=['TransactionID'], right_on=['TransactionID'], copy=False)
df_test = pd.merge(df_test, df_temp, how='left', left_on=['TransactionID'], right_on=['TransactionID'], copy=False)
df_valid = pd.merge(df_valid, df_temp, how='left', left_on=['TransactionID'], right_on=['TransactionID'], copy=False)


In [ ]:
# saving the data
project.save_data(data=df_train.to_csv(index=False),file_name='train_short_new.csv',overwrite=True)
print('train done')
project.save_data(data=df_valid.to_csv(index=False),file_name='valid_short_new.csv',overwrite=True)
print('valid done')
project.save_data(data=df_test.to_csv(index=False),file_name='test_short_new.csv',overwrite=True)

### Different services are characterised by different features like - average of specific type of product is very different that the same of different product category or hours of transaction are very different. There are 5 different types of products and I have generated features seperatly for these products. Similar code for other products (only Product 1 covered below)
### Card identifiers like card1 has large number of unique values. It has high feature importance in the final model however there are new cards as we advance in time. Below feature engineering also helps to make model independent of these variables. e.g. instead of grouping based on card, models groups the similar cards basis average transaction amount, count of transactions and other generated features. This helps to improve the performance on test data significantly. 

In [ ]:
df_train1 = df_train[df_train['ProductCD']==1]
df_test1 = df_test[df_test['ProductCD']==1]
df_valid1 = df_valid[df_valid['ProductCD']==1]

# Python does not free the memory back for usage. So as a better practice I assign the variable to 0 before deleting. Reassigning is better way to free up memory in python
df_train = 0
df_test = 0
df_valid = 0
del df_train
del df_valid
del df_test

# calculating mean values for different cards and uid. EDA for the generated features is to be carried out later for determining if the feture is to be retained.
# This notebook is to understand feature engineering techniques

df_temp = pd.concat([df_train1[['TransactionAmt','card1']],df_valid1[['TransactionAmt','card1']],df_test1[['TransactionAmt','card1']]],copy=False)
df_temp = df_temp[['TransactionAmt','card1']].groupby(['card1']).mean()
df_temp.rename(columns={'TransactionAmt':'card1_mean'}, inplace=True)
df_train1 = pd.merge(df_train1, df_temp, how='left', left_on=['card1'], right_on=['card1'], copy=False)
df_test1 = pd.merge(df_test1, df_temp, how='left', left_on=['card1'], right_on=['card1'], copy=False)
df_valid1 = pd.merge(df_valid1, df_temp, how='left', left_on=['card1'], right_on=['card1'], copy=False)

df_temp = pd.concat([df_train1[['TransactionAmt','card2']],df_valid1[['TransactionAmt','card2']],df_test1[['TransactionAmt','card2']]],copy=False)
df_temp = df_temp[['TransactionAmt','card2']].groupby(['card2']).mean()
df_temp.rename(columns={'TransactionAmt':'card2_mean'}, inplace=True)
df_train1 = pd.merge(df_train1, df_temp, how='left', left_on=['card2'], right_on=['card2'], copy=False)
df_test1 = pd.merge(df_test1, df_temp, how='left', left_on=['card2'], right_on=['card2'], copy=False)
df_valid1 = pd.merge(df_valid1, df_temp, how='left', left_on=['card2'], right_on=['card2'], copy=False)

df_temp = pd.concat([df_train1[['TransactionAmt','card3']],df_valid1[['TransactionAmt','card3']],df_test1[['TransactionAmt','card3']]],copy=False)
df_temp = df_temp[['TransactionAmt','card3']].groupby(['card3']).mean()
df_temp.rename(columns={'TransactionAmt':'card3_mean'}, inplace=True)
df_train1 = pd.merge(df_train1, df_temp, how='left', left_on=['card3'], right_on=['card3'], copy=False)
df_test1 = pd.merge(df_test1, df_temp, how='left', left_on=['card3'], right_on=['card3'], copy=False)
df_valid1 = pd.merge(df_valid1, df_temp, how='left', left_on=['card3'], right_on=['card3'], copy=False)

df_temp = pd.concat([df_train1[['TransactionAmt','card4']],df_valid1[['TransactionAmt','card4']],df_test1[['TransactionAmt','card4']]],copy=False)
df_temp = df_temp[['TransactionAmt','card4']].groupby(['card4']).mean()
df_temp.rename(columns={'TransactionAmt':'card4_mean'}, inplace=True)
df_train1 = pd.merge(df_train1, df_temp, how='left', left_on=['card4'], right_on=['card4'], copy=False)
df_test1 = pd.merge(df_test1, df_temp, how='left', left_on=['card4'], right_on=['card4'], copy=False)
df_valid1 = pd.merge(df_valid1, df_temp, how='left', left_on=['card4'], right_on=['card4'], copy=False)

df_temp = pd.concat([df_train1[['TransactionAmt','card5']],df_valid1[['TransactionAmt','card5']],df_test1[['TransactionAmt','card5']]],copy=False)
df_temp = df_temp[['TransactionAmt','card5']].groupby(['card5']).mean()
df_temp.rename(columns={'TransactionAmt':'card5_mean'}, inplace=True)
df_train1 = pd.merge(df_train1, df_temp, how='left', left_on=['card5'], right_on=['card5'], copy=False)
df_test1 = pd.merge(df_test1, df_temp, how='left', left_on=['card5'], right_on=['card5'], copy=False)
df_valid1 = pd.merge(df_valid1, df_temp, how='left', left_on=['card5'], right_on=['card5'], copy=False)

df_temp = pd.concat([df_train1[['TransactionAmt','card6']],df_valid1[['TransactionAmt','card6']],df_test1[['TransactionAmt','card6']]],copy=False)
df_temp = df_temp[['TransactionAmt','card6']].groupby(['card6']).mean()
df_temp.rename(columns={'TransactionAmt':'card6_mean'}, inplace=True)
df_train1 = pd.merge(df_train1, df_temp, how='left', left_on=['card6'], right_on=['card6'], copy=False)
df_test1 = pd.merge(df_test1, df_temp, how='left', left_on=['card6'], right_on=['card6'], copy=False)
df_valid1 = pd.merge(df_valid1, df_temp, how='left', left_on=['card6'], right_on=['card6'], copy=False)

df_temp = pd.concat([df_train1[['TransactionAmt','addr1']],df_valid1[['TransactionAmt','addr1']],df_test1[['TransactionAmt','addr1']]],copy=False)
df_temp = df_temp[['TransactionAmt','addr1']].groupby(['addr1']).mean()
df_temp.rename(columns={'TransactionAmt':'addr1_mean'}, inplace=True)
df_train1 = pd.merge(df_train1, df_temp, how='left', left_on=['addr1'], right_on=['addr1'], copy=False)
df_test1 = pd.merge(df_test1, df_temp, how='left', left_on=['addr1'], right_on=['addr1'], copy=False)
df_valid1 = pd.merge(df_valid1, df_temp, how='left', left_on=['addr1'], right_on=['addr1'], copy=False)

df_temp = pd.concat([df_train1[['TransactionAmt','uid']],df_valid1[['TransactionAmt','uid']],df_test1[['TransactionAmt','uid']]],copy=False)
df_temp = df_temp[['TransactionAmt','uid']].groupby(['uid']).mean()
df_temp.rename(columns={'TransactionAmt':'uid_mean'}, inplace=True)
df_train1 = pd.merge(df_train1, df_temp, how='left', left_on=['uid'], right_on=['uid'], copy=False)
df_test1 = pd.merge(df_test1, df_temp, how='left', left_on=['uid'], right_on=['uid'], copy=False)
df_valid1 = pd.merge(df_valid1, df_temp, how='left', left_on=['uid'], right_on=['uid'], copy=False)

df_temp = pd.concat([df_train1[['TransactionAmt','uid1']],df_valid1[['TransactionAmt','uid1']],df_test1[['TransactionAmt','uid1']]],copy=False)
df_temp = df_temp[['TransactionAmt','uid1']].groupby(['uid1']).mean()
df_temp.rename(columns={'TransactionAmt':'uid1_mean'}, inplace=True)
df_train1 = pd.merge(df_train1, df_temp, how='left', left_on=['uid1'], right_on=['uid1'], copy=False)
df_test1 = pd.merge(df_test1, df_temp, how='left', left_on=['uid1'], right_on=['uid1'], copy=False)
df_valid1 = pd.merge(df_valid1, df_temp, how='left', left_on=['uid1'], right_on=['uid1'], copy=False)


In [ ]:
# calculating the standard deviation for transaction amout 

df_temp = pd.concat([df_train1[['TransactionAmt','card1']],df_valid1[['TransactionAmt','card1']],df_test1[['TransactionAmt','card1']]],copy=False)
df_temp = df_temp[['TransactionAmt','card1']].groupby(['card1']).std(ddof=0) # To avoid nan values where count is 1
df_temp.rename(columns={'TransactionAmt':'card1_std'}, inplace=True)
df_train1 = pd.merge(df_train1, df_temp, how='left', left_on=['card1'], right_on=['card1'], copy=False)
df_test1 = pd.merge(df_test1, df_temp, how='left', left_on=['card1'], right_on=['card1'], copy=False)
df_valid1 = pd.merge(df_valid1, df_temp, how='left', left_on=['card1'], right_on=['card1'], copy=False)

df_temp = pd.concat([df_train1[['TransactionAmt','card2']],df_valid1[['TransactionAmt','card2']],df_test1[['TransactionAmt','card2']]],copy=False)
df_temp = df_temp[['TransactionAmt','card2']].groupby(['card2']).std(ddof=0)
df_temp.rename(columns={'TransactionAmt':'card2_std'}, inplace=True)
df_train1 = pd.merge(df_train1, df_temp, how='left', left_on=['card2'], right_on=['card2'], copy=False)
df_test1 = pd.merge(df_test1, df_temp, how='left', left_on=['card2'], right_on=['card2'], copy=False)
df_valid1 = pd.merge(df_valid1, df_temp, how='left', left_on=['card2'], right_on=['card2'], copy=False)

df_temp = pd.concat([df_train1[['TransactionAmt','card3']],df_valid1[['TransactionAmt','card3']],df_test1[['TransactionAmt','card3']]],copy=False)
df_temp = df_temp[['TransactionAmt','card3']].groupby(['card3']).std(ddof=0)
df_temp.rename(columns={'TransactionAmt':'card3_std'}, inplace=True)
df_train1 = pd.merge(df_train1, df_temp, how='left', left_on=['card3'], right_on=['card3'], copy=False)
df_test1 = pd.merge(df_test1, df_temp, how='left', left_on=['card3'], right_on=['card3'], copy=False)
df_valid1 = pd.merge(df_valid1, df_temp, how='left', left_on=['card3'], right_on=['card3'], copy=False)

df_temp = pd.concat([df_train1[['TransactionAmt','card4']],df_valid1[['TransactionAmt','card4']],df_test1[['TransactionAmt','card4']]],copy=False)
df_temp = df_temp[['TransactionAmt','card4']].groupby(['card4']).std(ddof=0)
df_temp.rename(columns={'TransactionAmt':'card4_std'}, inplace=True)
df_train1 = pd.merge(df_train1, df_temp, how='left', left_on=['card4'], right_on=['card4'], copy=False)
df_test1 = pd.merge(df_test1, df_temp, how='left', left_on=['card4'], right_on=['card4'], copy=False)
df_valid1 = pd.merge(df_valid1, df_temp, how='left', left_on=['card4'], right_on=['card4'], copy=False)

df_temp = pd.concat([df_train1[['TransactionAmt','card5']],df_valid1[['TransactionAmt','card5']],df_test1[['TransactionAmt','card5']]],copy=False)
df_temp = df_temp[['TransactionAmt','card5']].groupby(['card5']).std(ddof=0)
df_temp.rename(columns={'TransactionAmt':'card5_std'}, inplace=True)
df_train1 = pd.merge(df_train1, df_temp, how='left', left_on=['card5'], right_on=['card5'], copy=False)
df_test1 = pd.merge(df_test1, df_temp, how='left', left_on=['card5'], right_on=['card5'], copy=False)
df_valid1 = pd.merge(df_valid1, df_temp, how='left', left_on=['card5'], right_on=['card5'], copy=False)

df_temp = pd.concat([df_train1[['TransactionAmt','card6']],df_valid1[['TransactionAmt','card6']],df_test1[['TransactionAmt','card6']]],copy=False)
df_temp = df_temp[['TransactionAmt','card6']].groupby(['card6']).std(ddof=0)
df_temp.rename(columns={'TransactionAmt':'card6_std'}, inplace=True)
df_train1 = pd.merge(df_train1, df_temp, how='left', left_on=['card6'], right_on=['card6'], copy=False)
df_test1 = pd.merge(df_test1, df_temp, how='left', left_on=['card6'], right_on=['card6'], copy=False)
df_valid1 = pd.merge(df_valid1, df_temp, how='left', left_on=['card6'], right_on=['card6'], copy=False)

df_temp = pd.concat([df_train1[['TransactionAmt','addr1']],df_valid1[['TransactionAmt','addr1']],df_test1[['TransactionAmt','addr1']]],copy=False)
df_temp = df_temp[['TransactionAmt','addr1']].groupby(['addr1']).std(ddof=0)
df_temp.rename(columns={'TransactionAmt':'addr1_std'}, inplace=True)
df_train1 = pd.merge(df_train1, df_temp, how='left', left_on=['addr1'], right_on=['addr1'], copy=False)
df_test1 = pd.merge(df_test1, df_temp, how='left', left_on=['addr1'], right_on=['addr1'], copy=False)
df_valid1 = pd.merge(df_valid1, df_temp, how='left', left_on=['addr1'], right_on=['addr1'], copy=False)

df_temp = pd.concat([df_train1[['TransactionAmt','uid']],df_valid1[['TransactionAmt','uid']],df_test1[['TransactionAmt','uid']]],copy=False)
df_temp = df_temp[['TransactionAmt','uid']].groupby(['uid']).std(ddof=0)
df_temp.rename(columns={'TransactionAmt':'uid_std'}, inplace=True)
df_train1 = pd.merge(df_train1, df_temp, how='left', left_on=['uid'], right_on=['uid'], copy=False)
df_test1 = pd.merge(df_test1, df_temp, how='left', left_on=['uid'], right_on=['uid'], copy=False)
df_valid1 = pd.merge(df_valid1, df_temp, how='left', left_on=['uid'], right_on=['uid'], copy=False)

df_temp = pd.concat([df_train1[['TransactionAmt','uid1']],df_valid1[['TransactionAmt','uid1']],df_test1[['TransactionAmt','uid1']]],copy=False)
df_temp = df_temp[['TransactionAmt','uid1']].groupby(['uid1']).std(ddof=0)
df_temp.rename(columns={'TransactionAmt':'uid1_std'}, inplace=True)
df_train1 = pd.merge(df_train1, df_temp, how='left', left_on=['uid1'], right_on=['uid1'], copy=False)
df_test1 = pd.merge(df_test1, df_temp, how='left', left_on=['uid1'], right_on=['uid1'], copy=False)
df_valid1 = pd.merge(df_valid1, df_temp, how='left', left_on=['uid1'], right_on=['uid1'], copy=False)


In [ ]:
# saving data
project.save_data(data=df_train1.to_csv(index=False),file_name='train1_fe1_999.csv',overwrite=True)
print('train done')
project.save_data(data=df_valid1.to_csv(index=False),file_name='valid1_fe1_999.csv',overwrite=True)
print('valid done')
project.save_data(data=df_test1.to_csv(index=False),file_name='test1_fe1_999.csv',overwrite=True)

### Buying pattern is also identified by time of transaction. for e.g. grocerry shopping, office hours etc
### Below code identifies the men and standard deviation of the transaction time

In [ ]:
df_temp = pd.concat([df_train1[['hour','addr1']],df_valid1[['hour','addr1']],df_test1[['hour','addr1']]],copy=False)
df_temp = df_temp[['hour','addr1']].groupby(['addr1']).mean()
df_temp.rename(columns={'hour':'addr1_mean_hour'}, inplace=True)
df_train1 = pd.merge(df_train1, df_temp, how='left', left_on=['addr1'], right_on=['addr1'], copy=False)
df_test1 = pd.merge(df_test1, df_temp, how='left', left_on=['addr1'], right_on=['addr1'], copy=False)
df_valid1 = pd.merge(df_valid1, df_temp, how='left', left_on=['addr1'], right_on=['addr1'], copy=False)

df_temp = pd.concat([df_train1[['hour','addr1']],df_valid1[['hour','addr1']],df_test1[['hour','addr1']]],copy=False)
df_temp = df_temp[['hour','addr1']].groupby(['addr1']).std(ddof=0)
df_temp.rename(columns={'hour':'addr1_std_hour'}, inplace=True)
df_train1 = pd.merge(df_train1, df_temp, how='left', left_on=['addr1'], right_on=['addr1'], copy=False)
df_test1 = pd.merge(df_test1, df_temp, how='left', left_on=['addr1'], right_on=['addr1'], copy=False)
df_valid1 = pd.merge(df_valid1, df_temp, how='left', left_on=['addr1'], right_on=['addr1'], copy=False)

df_temp = pd.concat([df_train1[['hour','card1']],df_valid1[['hour','card1']],df_test1[['hour','card1']]],copy=False)
df_temp = df_temp[['hour','card1']].groupby(['card1']).mean()
df_temp.rename(columns={'hour':'card1_mean_hour'}, inplace=True)
df_train1 = pd.merge(df_train1, df_temp, how='left', left_on=['card1'], right_on=['card1'], copy=False)
df_test1 = pd.merge(df_test1, df_temp, how='left', left_on=['card1'], right_on=['card1'], copy=False)
df_valid1 = pd.merge(df_valid1, df_temp, how='left', left_on=['card1'], right_on=['card1'], copy=False)

df_temp = pd.concat([df_train1[['hour','card2']],df_valid1[['hour','card2']],df_test1[['hour','card2']]],copy=False)
df_temp = df_temp[['hour','card2']].groupby(['card2']).mean()
df_temp.rename(columns={'hour':'card2_mean_hour'}, inplace=True)
df_train1 = pd.merge(df_train1, df_temp, how='left', left_on=['card2'], right_on=['card2'], copy=False)
df_test1 = pd.merge(df_test1, df_temp, how='left', left_on=['card2'], right_on=['card2'], copy=False)
df_valid1 = pd.merge(df_valid1, df_temp, how='left', left_on=['card2'], right_on=['card2'], copy=False)

df_temp = pd.concat([df_train1[['hour','card3']],df_valid1[['hour','card3']],df_test1[['hour','card3']]],copy=False)
df_temp = df_temp[['hour','card3']].groupby(['card3']).mean()
df_temp.rename(columns={'hour':'card3_mean_hour'}, inplace=True)
df_train1 = pd.merge(df_train1, df_temp, how='left', left_on=['card3'], right_on=['card3'], copy=False)
df_test1 = pd.merge(df_test1, df_temp, how='left', left_on=['card3'], right_on=['card3'], copy=False)
df_valid1 = pd.merge(df_valid1, df_temp, how='left', left_on=['card3'], right_on=['card3'], copy=False)

df_temp = pd.concat([df_train1[['hour','card4']],df_valid1[['hour','card4']],df_test1[['hour','card4']]],copy=False)
df_temp = df_temp[['hour','card4']].groupby(['card4']).mean()
df_temp.rename(columns={'hour':'card4_mean_hour'}, inplace=True)
df_train1 = pd.merge(df_train1, df_temp, how='left', left_on=['card4'], right_on=['card4'], copy=False)
df_test1 = pd.merge(df_test1, df_temp, how='left', left_on=['card4'], right_on=['card4'], copy=False)
df_valid1 = pd.merge(df_valid1, df_temp, how='left', left_on=['card4'], right_on=['card4'], copy=False)

df_temp = pd.concat([df_train1[['hour','card5']],df_valid1[['hour','card5']],df_test1[['hour','card5']]],copy=False)
df_temp = df_temp[['hour','card5']].groupby(['card5']).mean()
df_temp.rename(columns={'hour':'card5_mean_hour'}, inplace=True)
df_train1 = pd.merge(df_train1, df_temp, how='left', left_on=['card5'], right_on=['card5'], copy=False)
df_test1 = pd.merge(df_test1, df_temp, how='left', left_on=['card5'], right_on=['card5'], copy=False)
df_valid1 = pd.merge(df_valid1, df_temp, how='left', left_on=['card5'], right_on=['card5'], copy=False)

df_temp = pd.concat([df_train1[['hour','card6']],df_valid1[['hour','card6']],df_test1[['hour','card6']]],copy=False)
df_temp = df_temp[['hour','card6']].groupby(['card6']).mean()
df_temp.rename(columns={'hour':'card6_mean_hour'}, inplace=True)
df_train1 = pd.merge(df_train1, df_temp, how='left', left_on=['card6'], right_on=['card6'], copy=False)
df_test1 = pd.merge(df_test1, df_temp, how='left', left_on=['card6'], right_on=['card6'], copy=False)
df_valid1 = pd.merge(df_valid1, df_temp, how='left', left_on=['card6'], right_on=['card6'], copy=False)

df_temp = pd.concat([df_train1[['hour','uid']],df_valid1[['hour','uid']],df_test1[['hour','uid']]],copy=False)
df_temp = df_temp[['hour','uid']].groupby(['uid']).mean()
df_temp.rename(columns={'hour':'uid_mean_hour'}, inplace=True)
df_train1 = pd.merge(df_train1, df_temp, how='left', left_on=['uid'], right_on=['uid'], copy=False)
df_test1 = pd.merge(df_test1, df_temp, how='left', left_on=['uid'], right_on=['uid'], copy=False)
df_valid1 = pd.merge(df_valid1, df_temp, how='left', left_on=['uid'], right_on=['uid'], copy=False)

df_temp = pd.concat([df_train1[['hour','uid1']],df_valid1[['hour','uid1']],df_test1[['hour','uid1']]],copy=False)
df_temp = df_temp[['hour','uid1']].groupby(['uid1']).mean()
df_temp.rename(columns={'hour':'uid1_mean_hour'}, inplace=True)
df_train1 = pd.merge(df_train1, df_temp, how='left', left_on=['uid1'], right_on=['uid1'], copy=False)
df_test1 = pd.merge(df_test1, df_temp, how='left', left_on=['uid1'], right_on=['uid1'], copy=False)
df_valid1 = pd.merge(df_valid1, df_temp, how='left', left_on=['uid1'], right_on=['uid1'], copy=False)


In [ ]:
df_temp = pd.concat([df_train1[['hour','card1']],df_valid1[['hour','card1']],df_test1[['hour','card1']]],copy=False)
df_temp = df_temp[['hour','card1']].groupby(['card1']).std(ddof=0)
df_temp.rename(columns={'hour':'card1_std_hour'}, inplace=True)
df_train1 = pd.merge(df_train1, df_temp, how='left', left_on=['card1'], right_on=['card1'], copy=False)
df_test1 = pd.merge(df_test1, df_temp, how='left', left_on=['card1'], right_on=['card1'], copy=False)
df_valid1 = pd.merge(df_valid1, df_temp, how='left', left_on=['card1'], right_on=['card1'], copy=False)

df_temp = pd.concat([df_train1[['hour','card2']],df_valid1[['hour','card2']],df_test1[['hour','card2']]],copy=False)
df_temp = df_temp[['hour','card2']].groupby(['card2']).std(ddof=0)
df_temp.rename(columns={'hour':'card2_std_hour'}, inplace=True)
df_train1 = pd.merge(df_train1, df_temp, how='left', left_on=['card2'], right_on=['card2'], copy=False)
df_test1 = pd.merge(df_test1, df_temp, how='left', left_on=['card2'], right_on=['card2'], copy=False)
df_valid1 = pd.merge(df_valid1, df_temp, how='left', left_on=['card2'], right_on=['card2'], copy=False)

df_temp = pd.concat([df_train1[['hour','card3']],df_valid1[['hour','card3']],df_test1[['hour','card3']]],copy=False)
df_temp = df_temp[['hour','card3']].groupby(['card3']).std(ddof=0)
df_temp.rename(columns={'hour':'card3_std_hour'}, inplace=True)
df_train1 = pd.merge(df_train1, df_temp, how='left', left_on=['card3'], right_on=['card3'], copy=False)
df_test1 = pd.merge(df_test1, df_temp, how='left', left_on=['card3'], right_on=['card3'], copy=False)
df_valid1 = pd.merge(df_valid1, df_temp, how='left', left_on=['card3'], right_on=['card3'], copy=False)

df_temp = pd.concat([df_train1[['hour','card4']],df_valid1[['hour','card4']],df_test1[['hour','card4']]],copy=False)
df_temp = df_temp[['hour','card4']].groupby(['card4']).std(ddof=0)
df_temp.rename(columns={'hour':'card4_std_hour'}, inplace=True)
df_train1 = pd.merge(df_train1, df_temp, how='left', left_on=['card4'], right_on=['card4'], copy=False)
df_test1 = pd.merge(df_test1, df_temp, how='left', left_on=['card4'], right_on=['card4'], copy=False)
df_valid1 = pd.merge(df_valid1, df_temp, how='left', left_on=['card4'], right_on=['card4'], copy=False)

df_temp = pd.concat([df_train1[['hour','card5']],df_valid1[['hour','card5']],df_test1[['hour','card5']]],copy=False)
df_temp = df_temp[['hour','card5']].groupby(['card5']).std(ddof=0)
df_temp.rename(columns={'hour':'card5_std_hour'}, inplace=True)
df_train1 = pd.merge(df_train1, df_temp, how='left', left_on=['card5'], right_on=['card5'], copy=False)
df_test1 = pd.merge(df_test1, df_temp, how='left', left_on=['card5'], right_on=['card5'], copy=False)
df_valid1 = pd.merge(df_valid1, df_temp, how='left', left_on=['card5'], right_on=['card5'], copy=False)

df_temp = pd.concat([df_train1[['hour','card6']],df_valid1[['hour','card6']],df_test1[['hour','card6']]],copy=False)
df_temp = df_temp[['hour','card6']].groupby(['card6']).std(ddof=0)
df_temp.rename(columns={'hour':'card6_std_hour'}, inplace=True)
df_train1 = pd.merge(df_train1, df_temp, how='left', left_on=['card6'], right_on=['card6'], copy=False)
df_test1 = pd.merge(df_test1, df_temp, how='left', left_on=['card6'], right_on=['card6'], copy=False)
df_valid1 = pd.merge(df_valid1, df_temp, how='left', left_on=['card6'], right_on=['card6'], copy=False)

df_temp = pd.concat([df_train1[['hour','uid']],df_valid1[['hour','uid']],df_test1[['hour','uid']]],copy=False)
df_temp = df_temp[['hour','uid']].groupby(['uid']).std(ddof=0)
df_temp.rename(columns={'hour':'uid_std_hour'}, inplace=True)
df_train1 = pd.merge(df_train1, df_temp, how='left', left_on=['uid'], right_on=['uid'], copy=False)
df_test1 = pd.merge(df_test1, df_temp, how='left', left_on=['uid'], right_on=['uid'], copy=False)
df_valid1 = pd.merge(df_valid1, df_temp, how='left', left_on=['uid'], right_on=['uid'], copy=False)

df_temp = pd.concat([df_train1[['hour','uid1']],df_valid1[['hour','uid1']],df_test1[['hour','uid1']]],copy=False)
df_temp = df_temp[['hour','uid1']].groupby(['uid1']).std(ddof=0)
df_temp.rename(columns={'hour':'uid1_std_hour'}, inplace=True)
df_train1 = pd.merge(df_train1, df_temp, how='left', left_on=['uid1'], right_on=['uid1'], copy=False)
df_test1 = pd.merge(df_test1, df_temp, how='left', left_on=['uid1'], right_on=['uid1'], copy=False)
df_valid1 = pd.merge(df_valid1, df_temp, how='left', left_on=['uid1'], right_on=['uid1'], copy=False)


In [ ]:
# checking for null values
df_null = df_test1.isna()
for col in df_test1.columns:
    if True in df_null[col].unique():
        print(col)
df_null = 0
del df_null

In [ ]:
# saving data
project.save_data(data=df_train1.to_csv(index=False),file_name='train1_fe2_999.csv',overwrite=True)
print('train done')
project.save_data(data=df_valid1.to_csv(index=False),file_name='valid1_fe2_999.csv',overwrite=True)
print('valid done')
project.save_data(data=df_test1.to_csv(index=False),file_name='test1_fe2_999.csv',overwrite=True)

### Steps forward
### 1. New features generated are merged back to the train and test data set.
### 2. EDA is performed on new fetures for retaining or deleting the feature. 

### Modeling, parameter tuning and feature selection will be published in next notebooks
